Perguntas

- O que a classe social (representada por Pclass) diz sobre as prioridades e o valor de uma vida humana naquela época?

- A idade e o gênero determinavam o direito à vida? Qual a relação entre a lógica dos dados e a ética do "mulheres e crianças primeiro"?

- O que os dados faltantes (missing values) nos dizem sobre o que não sabemos? Isso pode representar a incerteza e o caos de um evento como o naufrágio.



In [ ]:
import pandas as pd
import numpy as np

# path do dataset a ser analisado
df = pd.read_csv('../data/Titanic-Dataset.csv')

# Exibir as cinco primeiras linhas
df.head()

- Entendimento e Preparação dos dados

In [ ]:
# criacao de coluna "Title"
import re

# Cria uma nova coluna 'Title' extraindo o título do nome
df['Title'] = df['Name'].apply(
    lambda x: re.search('([A-Za-z]+)\.', x).group(1)
)
df.head()

In [ ]:
# Verificar frequencia dos titulos
title_counts = df['Title'].value_counts()
print(title_counts)

In [ ]:
# Agrupar os títulos raros em uma única categoria "Rare" para simplificar as analises

titulos_raros = [
    'Dr', 'Rev', 'Mlle', 'Major', 'Col', 'Countess', 'Capt',
    'Ms', 'Sir', 'Lady', 'Mme', 'Don', 'Jonkheer'
]

# Criar dicionário para o mapeamento
mapeamento_titulos = {titulo: 'Rare' for titulo in titulos_raros}
mapeamento_titulos['Mlle'] = 'Miss'
mapeamento_titulos['Ms'] = 'Miss'
mapeamento_titulos['Mme'] = 'Mrs'

# Aplica o mapeamento
df['Title'] = df['Title'].replace(mapeamento_titulos)

print(df['Title'].value_counts())

In [ ]:
# Valores ausentes por coluna
valores_ausentes = df.isnull().sum()

# Calcular a porcentagem de valores ausentes
porcentagem_ausentes = (valores_ausentes / len(df)) * 100

# Criar DataFrame para visualizar a contagem e a porcentagem
df_valores_ausentes = pd.DataFrame({
    'Contagem': valores_ausentes,
    'Porcentagem': porcentagem_ausentes
})

# Filtrar apenas as colunas com valores ausentes e ordenamos
print(df_valores_ausentes[df_valores_ausentes['Contagem'] > 0].sort_values(by='Porcentagem', ascending=False))

- Analise dos dados acima: 

- Cabin: : Quase 80% dos dados estão faltando, inferir sobre estas informações é inviavel. Então melhor descartar esta coluna.
- Age:: A idade é uma variavel crucial, 20% faltante é uma quantidade consideravel.
- Embarked: Menos de 1% de dados ausentes, dois valores estão faltando. Posso preencher com o valor mais frequente tranquilamente.

In [ ]:
# Preencher idades faltantes

df['Age'] = df['Age'].fillna(
    df.groupby('Title')['Age'].transform('median')
)
print(df['Age'].isnull().sum())

In [ ]:
# Preencher o Porto de Embarque

porto_mais_comum = df['Embarked'].mode()[0]

df['Embarked'] = df['Embarked'].fillna(porto_mais_comum)

print(df['Embarked'].isnull().sum())

 - Respondendo a primeira pergunta:

 O que a classe social (representada por Pclass) diz sobre as prioridades e o valor de uma vida humana naquela época?

In [ ]:
# Cria uma tabela cruzada para ver a contagem de sobreviventes por classe
tabela_pclass = pd.crosstab(df['Pclass'], df['Survived'])

# Adiciona os totais para cada linha e coluna
tabela_pclass['Total'] = tabela_pclass.sum(axis=1)

# Calcula a porcentagem de sobreviventes por classe
tabela_pclass['Sobrevivência %'] = round((tabela_pclass[1] / tabela_pclass['Total']) * 100, 2)

# Altera os nomes das colunas para melhor compreensão
tabela_pclass = tabela_pclass.rename(columns={0: 'Não Sobreviveu', 1: 'Sobreviveu'})

print(tabela_pclass)

Análise dos Dados:

A tabela mostra uma correlação inegável entre classe social e sobrevivência.

1ª Classe: 62.96% de sobrevivência.

2ª Classe: 47.28% de sobrevivência.

3ª Classe: Apenas 24.24% de sobrevivência.

Reflexão Filosófica:

Os dados revelam uma hierarquia de valor de vida. O acesso a uma cabine na 1ª classe não era apenas uma questão de conforto; era, literalmente, a diferença entre a vida e a morte. O acesso aos botes salva-vidas, a prioridade no resgate e até a localização das cabines (mais altas no navio) estavam todos correlacionados com a classe social.

O naufrágio do Titanic, que deveria ser um evento de destino cego, foi, na verdade, um reflexo brutal das desigualdades sociais. Os dados, frios e objetivos, confirmam que o valor de uma vida era, em grande parte, determinado pelo status social.

- Respondendo a segunda Pergunta:

A idade e o gênero determinavam o direito à vida? Qual a relação entre a lógica dos dados e a ética do "mulheres e crianças primeiro"?

In [ ]:
# Cria uma tabela cruzada para ver a contagem de sobreviventes por sexo
tabela_sexo = pd.crosstab(df['Sex'], df['Survived'])

# Adiciona os totais para cada linha e coluna
tabela_sexo['Total'] = tabela_sexo.sum(axis=1)

# Calcula a porcentagem de sobreviventes por sexo
tabela_sexo['Sobrevivência %'] = round((tabela_sexo[1] / tabela_sexo['Total']) * 100, 2)

# Altera os nomes das colunas para melhor compreensão
tabela_sexo = tabela_sexo.rename(columns={0: 'Não Sobreviveu', 1: 'Sobreviveu'})

print(tabela_sexo)

Análise dos Dados:

Mulheres: 74.2% de sobrevivência.

Homens: Apenas 18.89% de sobrevivência.

Reflexão Filosófica:

O dado é brutalmente direto: o gênero era um fator muito mais decisivo para a sobrevivência do que a classe social. A ordem de "mulheres e crianças primeiro" não foi apenas um ideal ético, mas uma política de resgate que se manifestou de forma inegável nos dados.

In [ ]:
# Cria um DataFrame apenas com passageiros do sexo masculino, fazendo uma cópia
homens_df = df[df['Sex'] == 'male'].copy()

In [ ]:
# Cria um DataFrame apenas com passageiros do sexo masculino
homens_df = df[df['Sex'] == 'male']

# Categoriza os homens em crianças (até 16 anos) e adultos
homens_df['Categoria'] = homens_df['Age'].apply(
    lambda age: 'Crianca' if age <= 16 else 'Adulto'
)

# Cria uma tabela cruzada para ver a sobrevivência de homens (crianças vs. adultos)
tabela_criancas_adultos = pd.crosstab(homens_df['Categoria'], homens_df['Survived'])

# Adiciona a porcentagem de sobrevivência
tabela_criancas_adultos['Sobrevivência %'] = round((tabela_criancas_adultos[1] / tabela_criancas_adultos.sum(axis=1)) * 100, 2)

# Altera os nomes das colunas
tabela_criancas_adultos = tabela_criancas_adultos.rename(columns={0: 'Não Sobreviveu', 1: 'Sobreviveu'})

print(tabela_criancas_adultos)

Análise dos Dados:

Homens Adultos: Apenas 16.28% de sobrevivência.

Meninos (Criança): 43.64% de sobrevivência.

Reflexão Filosófica:

Os números falam por si. A política de "mulheres e crianças primeiro" foi de fato aplicada. A sobrevivência dos meninos foi significativamente maior que a dos homens adultos. O que a ética prescrevia, a lógica dos dados confirmou: em uma situação de escassez (de botes salva-vidas), vidas foram priorizadas, e essa prioridade foi baseada na vulnerabilidade percebida

Em suma, a análise mostra que a idade e o gênero foram, de fato, os fatores determinantes para o direito à vida. O que parecia uma ordem ética e humanitária se revela uma regra estatística, com o valor de uma vida sendo quantificado pela sua idade e sexo.

- Respondendo a Terceira Pergunta:

O que os dados faltantes (missing values) nos dizem sobre o que não sabemos? Isso pode representar a incerteza e o caos de um evento como o naufrágio.

In [ ]:
# Contar o número de valores ausentes por coluna
valores_ausentes_finais = df.isnull().sum()

# Filtrar apenas as colunas com valores ausentes
df_valores_ausentes_finais = valores_ausentes_finais[valores_ausentes_finais > 0]

# Exibir o resultado
print("Valores ausentes após a limpeza:")
print(df_valores_ausentes_finais)

if df_valores_ausentes_finais.empty:
    print("\nO seu DataFrame está 100% limpo!")

In [ ]:
# Descarta a coluna 'Cabin'
df = df.drop('Cabin', axis=1, errors='ignore')

# Verificamos a nova contagem de valores ausentes para confirmar a remoção
print(df.isnull().sum())

Os valores ausentes na coluna Cabin mostra a incerteza e a falta de informação, levando à decisão pragmática de descartar a coluna. Já os poucos valores ausentes em Age e Embarked puderam ser preenchidos de forma inteligente, preservando o máximo de informação